In [33]:
max_vocab_size = 100000

training_split_ratio = 0.8

In [44]:
import torch
import torch.nn as nn

In [3]:
def load_text8_data(filepath):
    with open(file=filepath) as file:
        text = file.read()

    return text

In [6]:
text = load_text8_data(r"/home/askar/mlx/week1/team/MLX5W1T1/word2vec/data/text8-1mb")

In [8]:
text_words = text.split()

In [13]:
from collections import Counter

word_counts = Counter(text_words)

In [20]:
most_common_words = word_counts.most_common(max_vocab_size)

In [42]:
total_vocab_size = len(most_common_words)

In [24]:
most_common_words[0:10]

[('the', 10802),
 ('of', 6209),
 ('and', 4839),
 ('in', 3980),
 ('to', 3545),
 ('one', 3377),
 ('a', 3204),
 ('zero', 2387),
 ('is', 2052),
 ('nine', 1774)]

In [27]:
word_to_id = {word:id for id, (word, _) in enumerate(most_common_words)}

In [30]:
id_to_word = {id:word for word, id in word_to_id.items()}

In [37]:
traing_test_cutoff = int(len(text_words) * training_split_ratio)
training_words = text_words[:traing_test_cutoff]
test_words = text_words[traing_test_cutoff:]

In [39]:
print(f"""
Total dataset size: {len(text_words)}
Training dataset size: {len(training_words)}
Test dataset size: {len(test_words)}
      """)


Total dataset size: 175599
Training dataset size: 140479
Test dataset size: 35120
      


In [47]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_window=2) -> None:
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.linear = nn.Linear(in_features=embedding_dim, out_features=2*context_window*vocab_size)
        self.softmax = nn.Softmax(2*context_window*vocab_size)

    def forward(self, word_id):
        word_embed = self.embed(word_id)
        prediction = self.softmax(self.linear(word_embed))
        return prediction


In [48]:
import torch.nn.functional as F

In [60]:
def generate_labels(words, context_window):
    for i in range(context_window, len(words) - context_window):
        context_vectors = []
        for x in range(i-context_window, i+context_window):
            context_vectors.append(F.one_hot(torch.tensor(x), total_vocab_size))
        context = torch.cat(context_vectors)
        yield i, context


In [65]:
training_label_generator = generate_labels(training_words, 2)
test_label_generator = generate_labels(test_words, 2)

In [ ]:
epochs = 5

my_model = Word2Vec(total_vocab_size, 100)

for _ in range(epochs):
    pass